Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [5]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [6]:
top10

Index(['OTHER', 'FLUSHING-NORTH', 'UPPER EAST SIDE (59-79)',
       'UPPER EAST SIDE (79-96)', 'BEDFORD STUYVESANT', 'BOROUGH PARK',
       'UPPER WEST SIDE (59-79)', 'GRAMERCY', 'ASTORIA', 'FOREST HILLS'],
      dtype='object')

In [8]:
print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [9]:
df.describe()#BUILDING_CLASS_CATEGORY

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,0.0,23039.000000,23039.000000,23039.000000,23039.000000,2.303900e+04,23005.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,NaN,10791.186987,1.827423,0.286514,2.320891,3.508555e+03,1819.671724,1.624479,1.328133e+06
std,3713.269887,628.025255,NaN,1103.414210,10.090432,5.778224,12.070554,2.218610e+04,488.376571,0.803898,1.025394e+07
min,1.000000,1.000000,NaN,0.000000,0.000000,-148.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,NaN,10306.000000,0.000000,0.000000,1.000000,4.940000e+02,1920.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,NaN,11211.000000,1.000000,0.000000,1.000000,1.356000e+03,1940.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,NaN,11360.000000,2.000000,0.000000,2.000000,2.280000e+03,1965.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,NaN,11697.000000,750.000000,570.000000,755.000000,1.303935e+06,2019.000000,4.000000,8.500000e+08


In [14]:
condition = (df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS') & (df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)
df = df[condition]
print(df.shape)
df.head()

(3164, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [21]:
df['SALE_DATE'].describe()

count           3164
unique            91
top       01/31/2019
freq              78
Name: SALE_DATE, dtype: object

In [28]:

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')
train = df[df['SALE_DATE'] < cutoff]
test  = df[df['SALE_DATE'] >= cutoff]

In [31]:
train.shape

(2517, 21)

In [30]:
train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,2019-01-02


In [33]:
test.shape

(647, 21)

In [34]:
train.describe(exclude='number')#choose categaric column

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2517,2517,2517,2517,2517,2517,1,2517,2517,2517
unique,5,6,1,2,13,2507,1,889,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,117-45 125TH STREET,RP.,"4,000",A1,2019-01-31 00:00:00
freq,1209,2390,2517,2486,921,2,1,235,921,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [35]:
train['BOROUGH'].value_counts()

4    1209
5     664
3     401
2     242
1       1
Name: BOROUGH, dtype: int64

In [36]:
train['BOROUGH'].value_counts(normalize=True)

4    0.480334
5    0.263806
3    0.159317
2    0.096146
1    0.000397
Name: BOROUGH, dtype: float64

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(train['BOROUGH'])

/usr/local/lib/python3.6/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [40]:
target = 'SALE_PRICE'
high_cardinality = ['NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT','ADDRESS','APARTMENT_NUMBER','LAND_SQUARE_FEET','BUILDING_CLASS_AT_TIME_OF_SALE','SALE_DATE']
features = train.columns.drop([target] + high_cardinality)
features


Index(['BOROUGH', 'BLOCK', 'LOT', 'EASE-MENT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [41]:
train.shape, test.shape

((2517, 21), (647, 21))

In [60]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2517, 11), (647, 11), (2517,), (647,))

In [61]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [62]:
X_train.shape

(2517, 15)

In [63]:
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,5495,801,NaN,11230.0,1.0,0.0,1.0,1325.0,1930.0,1
61,0,1,0,0,0,7918,72,NaN,11427.0,1.0,0.0,1.0,2001.0,1940.0,1
78,0,0,1,0,0,4210,19,NaN,10461.0,1.0,0.0,1.0,2043.0,1925.0,1
108,1,0,0,0,0,5212,69,NaN,11226.0,1.0,0.0,1.0,2680.0,1899.0,1
111,1,0,0,0,0,7930,121,NaN,11203.0,1.0,0.0,1.0,1872.0,1940.0,1


In [69]:
X_train = X_train.drop(columns=['EASE-MENT'])

In [68]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 44 to 18147
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   BOROUGH_3                  2517 non-null   int64  
 1   BOROUGH_4                  2517 non-null   int64  
 2   BOROUGH_2                  2517 non-null   int64  
 3   BOROUGH_5                  2517 non-null   int64  
 4   BOROUGH_1                  2517 non-null   int64  
 5   BLOCK                      2517 non-null   int64  
 6   LOT                        2517 non-null   int64  
 7   EASE-MENT                  0 non-null      float64
 8   ZIP_CODE                   2517 non-null   float64
 9   RESIDENTIAL_UNITS          2517 non-null   int64  
 10  COMMERCIAL_UNITS           2517 non-null   float64
 11  TOTAL_UNITS                2517 non-null   float64
 12  GROSS_SQUARE_FEET          2517 non-null   float64
 13  YEAR_BUILT                 2517 non-null   flo

In [67]:
import numpy as np 
X_train['RESIDENTIAL_UNITS'] = X_train['RESIDENTIAL_UNITS'].apply(np.int64) 




In [70]:
X_train['ZIP_CODE'] = X_train['ZIP_CODE'].apply(np.int64)
X_train['COMMERCIAL_UNITS'] = X_train['COMMERCIAL_UNITS'].apply(np.int64)
X_train['TOTAL_UNITS'] = X_train['TOTAL_UNITS'].apply(np.int64)
X_train['GROSS_SQUARE_FEET'] = X_train['GROSS_SQUARE_FEET'].apply(np.int64)
X_train['YEAR_BUILT'] = X_train['YEAR_BUILT'].apply(np.int64)
X_train['TAX_CLASS_AT_TIME_OF_SALE'] = X_train['TAX_CLASS_AT_TIME_OF_SALE'].apply(np.int64)

In [71]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2517 entries, 44 to 18147
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   BOROUGH_3                  2517 non-null   int64
 1   BOROUGH_4                  2517 non-null   int64
 2   BOROUGH_2                  2517 non-null   int64
 3   BOROUGH_5                  2517 non-null   int64
 4   BOROUGH_1                  2517 non-null   int64
 5   BLOCK                      2517 non-null   int64
 6   LOT                        2517 non-null   int64
 7   ZIP_CODE                   2517 non-null   int64
 8   RESIDENTIAL_UNITS          2517 non-null   int64
 9   COMMERCIAL_UNITS           2517 non-null   int64
 10  TOTAL_UNITS                2517 non-null   int64
 11  GROSS_SQUARE_FEET          2517 non-null   int64
 12  YEAR_BUILT                 2517 non-null   int64
 13  TAX_CLASS_AT_TIME_OF_SALE  2517 non-null   int64
dtypes: int64(14)
memory us

In [77]:
X_test = X_test.drop(columns=['EASE-MENT'])
X_test['RESIDENTIAL_UNITS'] = X_test['RESIDENTIAL_UNITS'].apply(np.int64) 
X_test['ZIP_CODE'] = X_test['ZIP_CODE'].apply(np.int64)
X_test['COMMERCIAL_UNITS'] = X_test['COMMERCIAL_UNITS'].apply(np.int64)
X_test['TOTAL_UNITS'] = X_test['TOTAL_UNITS'].apply(np.int64)
X_test['GROSS_SQUARE_FEET'] = X_test['GROSS_SQUARE_FEET'].apply(np.int64)
X_test['YEAR_BUILT'] = X_test['YEAR_BUILT'].apply(np.int64)
X_test['TAX_CLASS_AT_TIME_OF_SALE'] = X_test['TAX_CLASS_AT_TIME_OF_SALE'].apply(np.int64)

In [78]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 647 entries, 18235 to 23035
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   BOROUGH_3                  647 non-null    int64
 1   BOROUGH_4                  647 non-null    int64
 2   BOROUGH_2                  647 non-null    int64
 3   BOROUGH_5                  647 non-null    int64
 4   BOROUGH_1                  647 non-null    int64
 5   BLOCK                      647 non-null    int64
 6   LOT                        647 non-null    int64
 7   ZIP_CODE                   647 non-null    int64
 8   RESIDENTIAL_UNITS          647 non-null    int64
 9   COMMERCIAL_UNITS           647 non-null    int64
 10  TOTAL_UNITS                647 non-null    int64
 11  GROSS_SQUARE_FEET          647 non-null    int64
 12  YEAR_BUILT                 647 non-null    int64
 13  TAX_CLASS_AT_TIME_OF_SALE  647 non-null    int64
dtypes: int64(14)
memory 

In [79]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(k=5)

X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 4 13] are constant.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [80]:
X_train.shape, X_train_selected.shape

((2517, 14), (2517, 5))

In [81]:
#  Which features were selected?

selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_1
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET
YEAR_BUILT


Features not selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
BLOCK
LOT
ZIP_CODE
RESIDENTIAL_UNITS
TAX_CLASS_AT_TIME_OF_SALE


In [83]:
#How many features should be selected?

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $185,788 

2 features
Test Mean Absolute Error: $186,656 

3 features
Test Mean Absolute Error: $185,097 

4 features
Test Mean Absolute Error: $185,607 

5 features
Test Mean Absolute Error: $178,918 

6 features
Test Mean Absolute Error: $168,465 

7 features
Test Mean Absolute Error: $167,926 

8 features
Test Mean Absolute Error: $169,357 

9 features
Test Mean Absolute Error: $169,357 

10 features
Test Mean Absolute Error: $169,422 

11 features
Test Mean Absolute Error: $169,433 

12 features
Test Mean Absolute Error: $169,465 

13 features
Test Mean Absolute Error: $169,465 

14 features
Test Mean Absolute Error: $169,465 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

In [97]:
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

def ridge_price(alpha):
  

    # Load data
    
    # Plot data
    ax = train.plot.scatter('GROSS_SQUARE_FEET', 'SALE_PRICE')

    # Fit linear model
    ridge = Ridge(alpha=alpha, normalize=True)
    ridge.fit(train[['GROSS_SQUARE_FEET']], train['SALE_PRICE'])

    # Get linear equation
    m = ridge.coef_[0].round(2)
    b = ridge.intercept_.round(2)
    title = f'Ridge Regression, alpha={alpha} \n y = {m}x + {b}'

    # Get predictions
    train['SALE_PRICE_pred'] = ridge.predict(train[['GROSS_SQUARE_FEET']])

    # Plot predictions
    train.plot('GROSS_SQUARE_FEET', 'SALE_PRICE_pred', ax=ax, title=title)
    plt.show()
    
    

ridge_price(alpha=0)

/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/core.py:328: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [85]:
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]

In [89]:
X_train.head()

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,5495,801,11230,1,0,1,1325,1930,1
61,0,1,0,0,0,7918,72,11427,1,0,1,2001,1940,1
78,0,0,1,0,0,4210,19,10461,1,0,1,2043,1925,1
108,1,0,0,0,0,5212,69,11226,1,0,1,2680,1899,1
111,1,0,0,0,0,7930,121,11203,1,0,1,1872,1940,1


In [90]:
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=alphas, normalize=True)
ridge.fit(train[['GROSS_SQUARE_FEET']], train['SALE_PRICE'])
ridge.alpha_

0.01

In [91]:
ax = train.plot.scatter('GROSS_SQUARE_FEET', 'SALE_PRICE')

# Get linear equation
m = ridge.coef_[0].round(2)
b = ridge.intercept_.round(2)
title = f'Ridge Regression, alpha={ridge.alpha_} \n y = {m}x + {b}'

# Get predictions
train['SALE_PRICE_pred'] = ridge.predict(train[['GROSS_SQUARE_FEET']])

# Plot predictions
train.plot('GROSS_SQUARE_FEET', 'SALE_PRICE_pred', ax=ax, title=title)
plt.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [87]:
for alpha in [0.001, 0.01, 0.1, 1.0, 1, 100.0, 1000.0]:
    
    # Fit Ridge Regression model
    display(HTML(f'Ridge Regression, with alpha={alpha}'))
    model = Ridge(alpha=alpha, normalize=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Get Test MAE
    mae = mean_absolute_error(y_test, y_pred)
    display(HTML(f'Test Mean Absolute Error: ${mae:,.0f}'))
    
    # Plot coefficients
    coefficients = pd.Series(model.coef_, X_train.columns)
    plt.figure(figsize=(16,8))
    coefficients.sort_values().plot.barh(color='grey')
    plt.xlim(-400,700)
    plt.show()

NameError: ignored